<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что сдала задание:) Ты проделала большую работу. Далее в файле мои комментарии ты сможешь найти в ячейках, аналогичных данфной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Количество заполненных строк в days_employed и total_income меньше на одинаковое количество. Незаполненные ячейки этих столбцов являются, скорее всего, знаком отсутствия стажа и дохода. Отсутствие трудового стажа и ежемесячного дохода можно прировнять к нулю. Заполним пропуски нулями.  

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Отличное и очень детально начало, так держать!)


### Шаг 2. Предобработка данных

### Обработка пропусков

Пропущенные значения мы обнаружили только в столбцах days_employed и total_income. 
Эти столбцы отражают трудовой стаж и ежемесячный доход. Отсутствие значений в них можно прировнять к нулю. 

In [2]:
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

Проверим результат

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Мы избавились от пропусков данных. 

<div style="border:solid orange 2px; padding: 20px"> <h1 style="color:orange; margin-bottom:20px">Комментарий наставника</h1>

Тут лучше не заполнять прпоуски нулём, стоило бы выбрать более обоснованный способ заполнения, иначе происходит искажение данных

### Замена типа данных

Из информации о таблице выше видно, что у нас есть несколько столбцов и вещественным типом данных. Это столбцы с трудовым стажем и ежемесячным доходом. Для нас не имеют значения часы и минуты трудового стажа, а так же копейки ежемесячного дохода. Для простоты вычислений нам необходимо привести данные к целочисленному типу. 

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

Мы заменили все вещественные типы данных на целочисленные. Из результатов видно, что теперь все числовые столбцы имеют тип int64. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Замена типа данных выполнена верно

### Обработка дубликатов

Проверим наличие дубликатов в таблице

In [5]:
print(data.duplicated().sum())

54


Мы обнаружили 54 дубликата в таблице. Удалим дубликаты. 

In [6]:
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())

0


Мы удалили основной тип дубликатов - полностью одинаковые строки. Проверим наличие дубликатов по уникальным значениям категорийных столбцов. 

In [7]:
print(data['education'].unique())
print(data['family_status'].unique())
print(data['income_type'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Из результатов выше видно, что дубликаты у нас есть в столбце education. Приведем все значения в нижний регистр методом str.lower()

In [8]:
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


### Вывод

По результатам проверки видно, что дубликатов в интересующем нас столбце больше нет. Все значения корректны и уникальны. В данном случае появление дубликатов - чисто человеческий фактор. Каждый может записать данные на свой лад. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Дубликаты найдены и обработаны верно,отлично


### Лемматизация

Посмотрим на уникальные значения в столбце с целями получения кредита. 

In [9]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Мы видим, что у нас есть цели, связанные с недвижимостью, образованием, автомобилем и прочим. Необходимо лемматизировать этот столбец. 

In [10]:
from pymystem3 import Mystem
m = Mystem()
def lemmatize_it(row):
        lemmas = m.lemmatize(row)
        if 'автомобиль' in lemmas:
            return 'автокредит'
        if 'свадьба' in lemmas:
            return 'потребительский'
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            return 'ипотека'
        if 'образование' in lemmas:
            return 'образование'
        
data['purpose_type'] = data['purpose'].apply(lemmatize_it)
print(data['purpose_type'])

0                ипотека
1             автокредит
2                ипотека
3            образование
4        потребительский
              ...       
21466            ипотека
21467         автокредит
21468            ипотека
21469         автокредит
21470         автокредит
Name: purpose_type, Length: 21471, dtype: object


Лемматизировали столбец, проверим уникальные значения в столбце с целями

In [11]:
print(data['purpose_type'].unique())

['ипотека' 'автокредит' 'образование' 'потребительский']


### Вывод

Мы привели все цели кредита к 4 типам. Теперь мы можем работать с этим столбцом. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация проведена верно

### Категоризация данных

Для дальнейшего анализа данных нам необходимо категоризировать следующие столбцы: количество детей, семейное положение, уровень дохода, цели кредита. Проверим уникальные значения в этих столбцах, чтобы понять, какие из них нуждаются в категоризации, а какие уже категоризированы. Я выбрала метод value_counts(), чтобы увидеть объем различных значений, а не только их присутствие, и исключить случаи, когда категория будет состоять из одного элемента. 

In [12]:
print('Значения количества детей:', data['children'].value_counts())
print('Значения семейного статуса:', data['family_status'].value_counts())
print('Значения ежемесячного дохода:', data['total_income'].value_counts())
print('Значения цели кредита:', data['purpose_type'].value_counts())

Значения количества детей:  0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
Значения семейного статуса: женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
Значения ежемесячного дохода: 0         2120
204827       3
163060       3
144533       3
121931       3
          ... 
109583       1
101387       1
138249       1
280240       1
229304       1
Name: total_income, Length: 18607, dtype: int64
Значения цели кредита: ипотека            10814
автокредит          4308
образование         4014
потребительский     2335
Name: purpose_type, dtype: int64


Мы видим, что столбцы с семейным положением и целями кредита уже не требуют категоризации. А вот столбцы с количеством детей и доходом нужно категоризировать. 
Так же мы видим, что у нас в столбце с количеством детей есть отрицательные значения (-1). Я считаю, что такое значение могло получиться в результате проставления тире перед значением при заполнении данных. Это значение нам нужно изменить на положительное. 

In [13]:
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts())

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Мы заменили отрицательные значения в количестве детей. Теперь распределим их по категориям: 0 - нет детей, 1-2 - малодетные, 3-20 - многодетные. Для этого создадим столбец child_type и напишем функцию, которая будет принимать значение количества детей и возвращать значение категории.

In [14]:
def child_type(row):
    if row == 0:
        return 'нет детей'
    elif row == 1 or row == 2:
        return 'малодетные'
    else:
        return 'многодетные'
data['child_type'] = data['children'].apply(child_type)
print(data['child_type'].value_counts())


нет детей      14107
малодетные      6908
многодетные      456
Name: child_type, dtype: int64


Мы категоризировали столбец с детьми. Теперь нужно категоризировать столбец с доходами. Посмотрим на минимальное, максимальное, среднее и медианные значения по доходам, чтобы определить категории.

In [15]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].mean())
print(data['total_income'].median())

0
2265604
150890.93591355783
135716.0


У нас есть нулевые значения, причислим их к категории "нет дохода". Значение медианы и среднего почти совпадают. Возьмем для оценки округленное среднее значение в 150000. Добавим категории "ниже среднего" и "выше среднего". Напишем функцию, которая будет принимать значение дохода и возвращать значение категории. 

In [16]:
def income_type(row):
    if row == 0:
        return 'нет дохода'
    elif row <= 150000:
        return 'ниже среднего'
    else:
        return 'выше среднего'
data['total_income_type'] = data['total_income'].apply(income_type)
print(data['total_income_type'].value_counts())

ниже среднего    10167
выше среднего     9184
нет дохода        2120
Name: total_income_type, dtype: int64


### Вывод

Мы категоризировали данные по всем необходимым нам столбцам. С такими данными можно работать. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Категоризация проведена верно

### Шаг 3. Ответьте на вопросы

Прежде, чем ответить на вопросы, создадим таблицу с нужными нам столбцами для удобства ее обработки. 

In [17]:
new_data = data[['child_type', 'family_status', 'total_income_type', 'purpose_type', 'debt']]
print(new_data)

        child_type     family_status total_income_type     purpose_type  debt
0       малодетные   женат / замужем     выше среднего          ипотека     0
1       малодетные   женат / замужем     ниже среднего       автокредит     0
2        нет детей   женат / замужем     ниже среднего          ипотека     0
3      многодетные   женат / замужем     выше среднего      образование     0
4        нет детей  гражданский брак     выше среднего  потребительский     0
...            ...               ...               ...              ...   ...
21466   малодетные  гражданский брак     выше среднего          ипотека     0
21467    нет детей   женат / замужем     выше среднего       автокредит     0
21468   малодетные  гражданский брак     ниже среднего          ипотека     1
21469  многодетные   женат / замужем     выше среднего       автокредит     1
21470   малодетные   женат / замужем     ниже среднего       автокредит     0

[21471 rows x 5 columns]


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем таблицу по столбцу child_type и выведем показатель debt

In [38]:
child_type = new_data.groupby(['child_type','debt']).agg({'debt':'count'})
print(child_type)

                   debt
child_type  debt       
малодетные  0      6269
            1       639
многодетные 0       417
            1        39
нет детей   0     13044
            1      1063


### Вывод

Из таблицы выше видно:
Среди заемщиков без детей процент задолженностей по кредиту равен 8%.
Среди заемщиков с малым количеством детей процент задолженностей равен: 10%.
Среди многодетных заемщиков процент задолженностей равен: 9%. 
Из этого можно сделать вывод, что наиболее надежными являются заемщики без детей, при этом отличия в процентах не такие большие, чтобы уверенно отказывать заемщикам с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
family_status = new_data.groupby(['family_status','debt']).agg({'debt':'count'})
print(family_status)

                             debt
family_status         debt       
Не женат / не замужем 0      2536
                      1       274
в разводе             0      1110
                      1        85
вдовец / вдова        0       896
                      1        63
гражданский брак      0      3775
                      1       388
женат / замужем       0     11413
                      1       931


### Вывод

Из таблицы выше видно:
Среди заемщиков, которые не женаты/не замужем, процент задолженностей по кредиту равен: 10%.
Среди заемщиков, которые в разводе, процент задолженностей по кредиту равен: 7%.
Среди вдов/вдовцов процент задолженностей по кредиту равен: 7%.
Среди заемщиков, состоящих в гражданском браке, процент задолженностей по кредиту равен: 10%.
Среди заемщиков, состоящих в браке, процент задолженностей по кредиту равен: 8%. 
Исходя из этого можно сделать вывод, что наиболее надежными являются вдовы/вдовцы или состоящие в разводе заемщики, а также заемщики, состоящие в браке. 
А наименее надежными ялвяются заемщики, которые не состоят в браке или состоят в гражданском браке. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
total_income_type = new_data.groupby(['total_income_type','debt']).agg({'debt':'count'})
print(total_income_type)

                        debt
total_income_type debt      
выше среднего     0     8458
                  1      726
нет дохода        0     1950
                  1      170
ниже среднего     0     9322
                  1      845


### Вывод

Из таблицы выше видно:
Среди заемщиков, доход которых выше среднего, процент задолженностей по кредиту равен: 8%.
Среди заемщиков с доходом ниже средного процент задолженностей равен: 8%. 
Среди заемщиков без дохода процент задолженностей равен: 9%. 
Исходя из этого можно сделать вывод, что уровень дохода не влияет на вероятность возникновения задолженности, а вот заемщики без дохода являются чуть менее надежными. 

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
purpose_type = new_data.groupby(['purpose_type','debt']).agg({'debt':'count'})
print(purpose_type)

                       debt
purpose_type    debt       
автокредит      0      3905
                1       403
ипотека         0     10032
                1       782
образование     0      3644
                1       370
потребительский 0      2149
                1       186


### Вывод

Процент задолженностей по целям кредита распределяется так:
Автокредит: 10%.
Ипотека: 7%.
Образование: 10%.
Потребительский: 8%. 
Самые надежные цели по кредиту - ипотека и потребительский. Скорее всего, это связано с тем, что суммы потребительских кредитов относительно невысокие, а в ипотеке заемщики рискуют своим собственным жильем, которое является залогом. А вот автокредит и кредит на образование являются самыми ненадежными с точки зрения возможности возникновения задолженностей со стороны заемщиков. 

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Выводы все правильные, молодец, но задание лучше было бы выполнить используя сводные таблицы


### Шаг 4. Общий вывод

Исходя из имеющихся у нас данных, можно сделать вывод, что банку при выдаче кредитов необходимо обращать особое внимание на потенциальных заемщиков, которые: не имеют постоянного дохода или целью получения кредита которых является приобретение автомобиля или получение образования. 

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Обязательно надо писать подробный вывод из проведённого анализа, в котором отразить все полученные результаты + дать советы по использованию данного анализа


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усердна и у тебя всё получится! Удачи в будущих проектах)